In [22]:
%matplotlib inline
import numpy as np
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Dense, \
    MaxPooling2D, \
    Convolution2D, \
    Dropout, \
    BatchNormalization, \
    ZeroPadding2D, \
    Lambda, \
    Flatten
from keras.preprocessing import image
import matplotlib.pyplot as plt

## Simple Linear Model

### Create Dummy Dataset

In [13]:
n = 1000
p = 3
X = np.random.rand(n, p)
w = np.array([5, 7, 2])
y = np.dot(X, w)

### Fit Model

In [46]:
model = Sequential([Dense(1, input_dim=3)])
model.compile(optimizer='sgd', loss='mean_squared_error')
model.fit(X, y, nb_epoch=100)

Epoch 1/100
1000/1000 [==============================] - 0s - loss: 25.1412     
Epoch 2/100
1000/1000 [==============================] - 0s - loss: 4.4256     
Epoch 3/100
1000/1000 [==============================] - 0s - loss: 2.2725     
Epoch 4/100
1000/1000 [==============================] - 0s - loss: 1.9321     
Epoch 5/100
1000/1000 [==============================] - 0s - loss: 1.7843     
Epoch 6/100
1000/1000 [==============================] - 0s - loss: 1.6588     
Epoch 7/100
1000/1000 [==============================] - 0s - loss: 1.5427     
Epoch 8/100
1000/1000 [==============================] - 0s - loss: 1.4380     
Epoch 9/100
1000/1000 [==============================] - 0s - loss: 1.3400     
Epoch 10/100
1000/1000 [==============================] - 0s - loss: 1.2498     
Epoch 11/100
1000/1000 [==============================] - 0s - loss: 1.1663     
Epoch 12/100
1000/1000 [==============================] - 0s - loss: 1.0891     
Epoch 13/100
1000/1000 [============

In [47]:
model.layers[0].get_weights()

[array([[ 4.87689924],
        [ 6.86038113],
        [ 1.86813855]], dtype=float32), array([ 0.20528407], dtype=float32)]

## DogsVsCats Linear Model using VGG outputs

In [2]:
train_path = '/mnt/data/dogscats/sample/train/'
valid_path = '/mnt/data/dogscats/sample/valid/'
vgg_mean = np.array([123.68, 116.779, 103.939], dtype=np.float32).reshape((3, 1, 1))

In [3]:
def get_batches(path, batch_size=4, idg=image.ImageDataGenerator(), shuffle=True, class_mode='categorical'):
    return idg.flow_from_directory(path, batch_size=batch_size, target_size=(224, 224), 
                                   shuffle=shuffle, class_mode=class_mode)

In [17]:
def addConvBlock(model, filters, layers):
    for layer in range(layers):
        model.add(ZeroPadding2D(padding=(1, 1)))
        model.add(Convolution2D(nb_filter=filters, nb_col=3, nb_row=3, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

In [24]:
def addFCBlock(model):
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))

In [19]:
def pre_process(x):
    x = x - vgg_mean
    return x[:, ::-1]

In [25]:
def create():
    model = Sequential()
    model.add(Lambda(pre_process, input_shape=(3, 224, 224), output_shape=(3,224,224)))
    addConvBlock(model, 2, 64)
    addConvBlock(model, 2, 128)
    addConvBlock(model, 3, 256)
    addConvBlock(model, 3, 512)
    addConvBlock(model, 3, 512)
    model.add(Flatten())
    addFCBlock(model)
    addFCBlock(model)
    model.add(Dense(1000, activation='softmax'))
    return model

In [26]:
model = create()

In [27]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_7 (Lambda)                (None, 3, 224, 224)   0           lambda_input_6[0][0]             
____________________________________________________________________________________________________
zeropadding2d_2945 (ZeroPadding2 (None, 3, 226, 226)   0           lambda_7[0][0]                   
____________________________________________________________________________________________________
convolution2d_2945 (Convolution2 (None, 2, 224, 224)   56          zeropadding2d_2945[0][0]         
____________________________________________________________________________________________________
zeropadding2d_2946 (ZeroPadding2 (None, 2, 226, 226)   0           convolution2d_2945[0][0]         
___________________________________________________________________________________________